# 二维卷积层
## 二维互相关运算
在二维卷积层中，一个二维输⼊入数组和一个二维核（kernel）数组通过互相关运算输出一个二维数组。  
![互相关运算](ExampleImages/互相关运算.png)
使用代码函数corr2d实现该运算

In [1]:
import torch
from torch import nn

def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i: i + h, j : j + w] * K).sum()
            
    return Y

In [2]:
# 构造数组验证运算
X = torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
K = torch.tensor([[0, 1], [2, 3]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [29]:
# 二维卷积层
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super(Conv2D, self).__init__()
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.randn(1))
        
    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

## 图像中物体边缘检测
检测图像中物体的边缘，即找到像素变化的位置。首先我们构造一张 的图像（即高和宽分别为6像素和8像素的图像）。它中间4列为黑（0），其余为（1）。

In [4]:
X = torch.ones(6, 8)
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

构造一个高和宽分别为1和2的卷积核K，当它与输入做互相关运算时，如果横向相邻元素相同，输出为0；否则输出为非0。

In [5]:
K = torch.tensor([[1, -1]])

In [6]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

## 通过数据学习核数组
使用物体边缘检测中的输入数据 X 和输出数据 Y 来学习我们构造的核数组K。我们首先构造一个卷积层，其卷积核将被初始化成随机数组。接下来在每一次迭代中，我们使用平方误差来比较 Y 和卷积层的输出，然后计算梯度来更新权重。

In [34]:
# 构造一个核数组形状是(1, 2)的二维卷积层
conv2d = Conv2D(kernel_size=(1, 2))

step = 30
lr = 0.01

for i in range(step):
    Y_hat = conv2d(X)
    l = ((Y_hat - Y) ** 2).sum()
    l.backward()
    
    # 梯度下降
    conv2d.weight.data -= lr * conv2d.weight.grad
    conv2d.bias.data -= lr * conv2d.bias.grad
    
    # 梯度清零
    conv2d.weight.grad.fill_(0)
    conv2d.bias.grad.fill_(0)
    
    if (i + 1) % 5 == 0:
        print('Step %d, loss %.3f' % (i + 1, l.item()))

Step 5, loss 4.186
Step 10, loss 1.165
Step 15, loss 0.324
Step 20, loss 0.090
Step 25, loss 0.025
Step 30, loss 0.007


In [35]:
print('weight:', conv2d.weight.data)
print('bias:', conv2d.bias.data)

weight: tensor([[ 0.9788, -0.9787]])
bias: tensor([-3.9233e-05])


# 填充和步幅
卷积层的输出形状由输入形状和卷积核窗口形状决定，填充和步幅是两个超参数，它们可以对给定形状的输入和卷积核改变输出的形状。

## 填充
填充（padding）是指在输入高和宽的两侧填充元素（通常是0元素）。
![填充](ExampleImages/填充.png)  
创建⼀个高和宽为3的二维卷积层，然后设输入高和宽两侧的填充数分别为1。给定一个高和宽为8的输入，我们发现输出的高和宽也是8。

In [36]:
def comp_conv2d(conv2d, X):
    # (1, 1)代表批量大小和通道数均为1
    X = X.view((1, 1) + X.shape)
    Y = conv2d(X)
    # 排除不关心的前两维：批量和通道
    return Y.view(Y.shape[2:])
# 两侧分别填充1行或列
conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, padding=1)

X = torch.rand(8, 8)
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

当卷积核的高和宽不同时，我们也可以通过设置高和宽上不同的填充数使输出和输入具有相同的高和宽。

In [37]:
# 使用高为5、宽为3的卷积核，在高和宽两侧的填充数分别为2和1
conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

## 步幅
每次滑动的行数和列数称为步幅（stride）。

In [38]:
# 领高和宽上的步幅均为2，从而使输入的高和宽减半
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

# 多输入通道和多输出通道
彩色图像在高和宽2个维度外还有RGB（红、绿、蓝）3个颜色通道。假设彩色图像的高和宽分别是h 和w（像素），那么它可以表示为一个3*h*w的多维数组。我们将大小为3的这一维称为通道（channel）维。


## 多输入通道
![两个通道的互相关](ExampleImages/2个输入通道的互相关计算.png)
实现含多个输入通道的互相关运算，只需要对每个通道做互相关预算，然后通过add_n函数进行累加。

In [1]:
import torch
from torch import nn
import sys
import d2lzh_pytorch as d2l

def corr2d_multi_in(X, K):
    # 沿着X和K的第0维（通道维）分别计算再相加
    res = d2l.corr2d(X[0, :, :], K[0, :, :])
    for i in range(1, X.shape[0]):
        res += d2l.corr2d(X[i, :, :], K[i, :, :])
    return res

构造输入数组X，核数组K来验证

In [19]:
X = torch.tensor([[[0, 1, 2], [3, 4, 5], [6, 7, 8]],
                 [[1, 2, 3], [4, 5, 6], [7, 8, 9]]])
K = torch.tensor([[[0, 1], [2, 3]], [[1, 2], [3, 4]]])

corr2d_multi_in(X, K)

[tensor([[0, 1],
        [2, 3]]), tensor([[1, 2],
        [3, 4]])]


## 多输出通道


In [20]:
def corr2d_multi_in_out(X, K):
    # 对k的第0维遍历，每次同输入X做互相关计算，所有结果使用stack函数合并在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K])

[tensor([[0, 1],
        [2, 3]]), tensor([[1, 2],
        [3, 4]])]


In [21]:
K = torch.stack([K, K + 1, K + 2])

corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

## $1\times1$卷积层
![1*1卷积层](ExampleImages/11卷积层.png)
使用全连接层中的矩阵乘法来实现该卷积层，这里需要矩阵乘法运算前后对数据形状做一些调整

In [25]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.view(c_i, h * w)
    K = K.view(c_o, c_i)
    # 全连接层的矩阵乘法
    Y = torch.mm(K, X)
    return Y.view(c_o, h, w)

In [26]:
# 验证
X = torch.rand(3, 3, 3)
K = torch.rand(2, 3, 1, 1)

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)

(Y1 - Y2).norm().item() < 1e-6

True

# 池化层
池化（pooling）层，它的提出是为了缓解卷积层对位置的过度敏感性。
## 二维最大池化层和平均池化层
不同于卷积层⾥里计算输入和核的互相关性，池化层直接计算池化窗口内元素的最大值或者平值。
![池化层](ExampleImages/池化层.png)

In [27]:
import torch
from torch import nn

def pool2d(X, pool_size, mode='max'):
    X = X.float()
    p_h, p_w = pool_size
    Y = torch.zeros(X.shape[0] - p_h + 1, X.shape[1] - p_w + 1)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode=='max':
                Y[i, j] = X[i:i + p_h, j:j + p_w].max()
            elif mode=='avg':
                Y[i, j] = X[i:i + p_h, j:j + p_w].mean()
                
    return Y

In [28]:
# 验证输出
X = torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
pool2d(X, (2, 2))

tensor([[4., 5.],
        [7., 8.]])

## 填充和步幅
同卷积层一样，池化层也可以在输入的高和宽两侧的填充并调整窗口的移动步幅来改变输出形状。池化层填充和步幅与卷积层填充和步幅的工作机制一样。

In [29]:
X = torch.arange(16, dtype=torch.float).view(1, 1, 4, 4)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

使用nn模块里的二维最大池化层MaxPool2d，该方法里步幅和池化窗口形状相同

In [30]:
pool2d = nn.MaxPool2d(3)
pool2d(X)

tensor([[[[10.]]]])

手动指定步幅和填充

In [31]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

## 多通道
在处理多通道输入数据时，池化层对每个输入通道分别池化，⽽不是像卷积层那样将各通道的输入按通道相加。这意味着池化层的输出通道数与输入通道数相等。

In [32]:
X = torch.cat((X, X + 1), dim=1)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [33]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])